Some additional experiments that didn't end up making it to the paper

In [ ]:
import sys  
sys.path.insert(0, '../src')
sys.path.insert(0, '../../meme/src')
sys.path.insert(0, '../../comparing-corpora/src')

import numpy as np
import pandas as pd
from compcor.text_tokenizer_embedder import STTokenizerEmbedder
from compcor.corpus_metrics import *
import matplotlib.pyplot as plt
import seaborn as sns
import os
from utils import QUORA, load_paraphrases, common_tokens_matrix, sort_numsuffix
from binarize_metrics import *
from ksc_methods import plotKSC, runKSC, runKSC_fixed_sample

In [ ]:
# generate base samples from QUORA of size subsample_size to ensure that C_A and C_B are paired paraphrases
np.random.seed(42)
subsample_size = 50
nrepetitions = 5 # number of random samplings of entire KSC set

res_out_dir = os.path.join(os.getcwd(), 'ksc_results', 'data')
fig_out_dir = os.path.join(os.getcwd(), 'ksc_results', 'figures')

# generate a single sampling of subsample_size pairs of paraphrases, to use for the KSC corpora
corpus1, corpus2 = load_paraphrases(subsample=subsample_size)
print(corpus1[:5])
print(corpus2[:5])

In [ ]:
# Some experiments with binarization of the distances to 0-1
# using cosine and pre-computed distance metric (submit clf=None)
classifier_distance_cos_mat = lambda c1, c2: classifier_distance_clf(corpus1=c1, corpus2=c2, clf=None)
PR1_cos_mat =  lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=1)
PR2_cos_mat =  lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=2)
PR3_cos_mat =  lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=3)
PR5_cos_mat =  lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=5)
PR10_cos_mat = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=10)

DC1_cos_mat =  lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=1)
DC2_cos_mat =  lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=2)
DC3_cos_mat =  lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=3)
DC5_cos_mat =  lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=5)
DC10_cos_mat = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=None, cosine=True, nearest_k=10)
IRPR_cos_mat = lambda c1, c2: IRPR_distance_clf(corpus1=c1, corpus2=c2, clf=None)
Hausdorff_cos_mat = lambda c1, c2: Directed_Hausdorff_distance_clf(corpus1=c1, corpus2=c2, clf=None)
Energy_cos_mat = lambda c1, c2: Energy_distance_clf(corpus1=c1, corpus2=c2, clf=None)

# cosine and other metrics without model (use TF-IDF encoding)
# using cosine rather than euclidean
PR1_cos_nomod =  lambda c1, c2: pr_distance(corpus1=c1, corpus2=c2, nearest_k=1, cosine=True, model=None)
PR2_cos_nomod =  lambda c1, c2: pr_distance(corpus1=c1, corpus2=c2, nearest_k=2, cosine=True, model=None)
PR3_cos_nomod =  lambda c1, c2: pr_distance(corpus1=c1, corpus2=c2, nearest_k=3, cosine=True, model=None)
PR5_cos_nomod =  lambda c1, c2: pr_distance(corpus1=c1, corpus2=c2, nearest_k=5, cosine=True, model=None)
PR10_cos_nomod = lambda c1, c2: pr_distance(corpus1=c1, corpus2=c2, nearest_k=10, cosine=True, model=None)

DC1_cos_nomod =  lambda c1, c2: dc_distance(corpus1=c1, corpus2=c2, nearest_k=1, cosine=True, model=None)
DC2_cos_nomod =  lambda c1, c2: dc_distance(corpus1=c1, corpus2=c2, nearest_k=2, cosine=True, model=None)
DC3_cos_nomod =  lambda c1, c2: dc_distance(corpus1=c1, corpus2=c2, nearest_k=3, cosine=True, model=None)
DC5_cos_nomod =  lambda c1, c2: dc_distance(corpus1=c1, corpus2=c2, nearest_k=5, cosine=True, model=None)
DC10_cos_nomod = lambda c1, c2: dc_distance(corpus1=c1, corpus2=c2, nearest_k=10, cosine=True, model=None)
Hausdorff_distance_nomod = lambda c1, c2: Directed_Hausdorff_distance(corpus1=c1, corpus2=c2, model=None)
Energy_distance_nomod = lambda c1, c2: Energy_distance(corpus1=c1, corpus2=c2, model=None)
fid_distance_nomod = lambda c1, c2: fid_distance(corpus1=c1, corpus2=c2, model=None)
mauve_distance_nomod = lambda c1, c2: mauve_distance(corpus1=c1, corpus2=c2, model=None)
classifier_distance_nomod = lambda c1, c2: classifier_distance(corpus1=c1, corpus2=c2, model=None)
IRPR_distance_nomod = lambda c1, c2: IRPR_distance(corpus1=c1, corpus2=c2, model=None)
medoid_distance_nomod = lambda c1, c2: medoid_distance(corpus1=c1, corpus2=c2, model=None)
median_distance_nomod = lambda c1, c2: median_distance(corpus1=c1, corpus2=c2, model=None)

# using cosine and binary modification to make it semantic or not
# FIT a SVC to the sampled corpora pairwise document distance to differentiate between paraphrase (or identical) distances and not
semantic_clf = cosine_clf(corpus1, corpus2)
# feed this fit classifier into the other functions
classifier_cos_semantic = lambda c1, c2: classifier_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf)
PR1_cos_semantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=1)
PR2_cos_semantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=2)
PR3_cos_semantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=3)
PR5_cos_semantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=5)
PR10_cos_semantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=10)
DC1_cos_semantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=1)
DC2_cos_semantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=2)
DC3_cos_semantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=3)
DC5_cos_semantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=5)
DC10_cos_semantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf, cosine=True, nearest_k=10)
IRPR_cos_semantic = lambda c1, c2: IRPR_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf)
Hausdorff_cos_semantic = lambda c1, c2: Directed_Hausdorff_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf)
Energy_cos_semantic = lambda c1, c2: Energy_distance_clf(corpus1=c1, corpus2=c2, clf=semantic_clf)

# non-semantic: a classifer that just returns 0 if distances are 0 and 1 if not (i.e. doesn't understand that paraphrases that are 
# not exactly equal should have 0 distance when binarized)
# doesn't need to be fit on distances
nonsemantic_clf = NonsemanticPred()
# feed this fit classifier into the other functions
classifier_cos_nonsemantic = lambda c1, c2: classifier_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf)
PR1_cos_nonsemantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=1)
PR2_cos_nonsemantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=2)
PR3_cos_nonsemantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=3)
PR5_cos_nonsemantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=5)
PR10_cos_nonsemantic = lambda c1, c2: pr_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=10)
DC1_cos_nonsemantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=1)
DC2_cos_nonsemantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=2)
DC3_cos_nonsemantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=3)

DC5_cos_nonsemantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=5)
DC10_cos_nonsemantic = lambda c1, c2: dc_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf, cosine=True, nearest_k=10)
IRPR_cos_nonsemantic = lambda c1, c2: IRPR_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf)
Hausdorff_cos_nonsemantic = lambda c1, c2: Directed_Hausdorff_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf)
Energy_cos_nonsemantic = lambda c1, c2: Energy_distance_clf(corpus1=c1, corpus2=c2, clf=nonsemantic_clf)


base_metrics_names = ['FID', 'MAUVE', 'CLASSIFIER', 'IRPR', 'DC_1','DC_2', 'DC_3','DC_5', 'DC_10', 'PR_1', 'PR_2', 'PR_3', 'PR_5', 'PR_10', 'HAUSDORFF', 'ENERGY', 'MEANDIST', 'MEDIANDIST']
base_nonsemantic_metrics = [chi_square_distance, zipf_distance]
base_nonsemantic_metrics_names = ['CHISQUARE', 'ZIPF']

base_metrics_nomod = [fid_distance_nomod, mauve_distance_nomod, classifier_distance_nomod, IRPR_distance_nomod,
               DC1_cos_nomod, DC2_cos_nomod, DC3_cos_nomod, DC5_cos_nomod, DC10_cos_nomod, PR1_cos_nomod, PR2_cos_nomod, PR3_cos_nomod, PR5_cos_nomod, PR10_cos_nomod, Hausdorff_distance_nomod, Energy_distance_nomod, medoid_distance_nomod, median_distance_nomod]
base_metrics_names_nomod = ['FID', 'MAUVE', 'CLASSIFIER', 'IRPR', 'DC_1','DC_2', 'DC_3','DC_5', 'DC_10', 'PR_1', 'PR_2', 'PR_3', 'PR_5', 'PR_10', 'HAUSDORFF', 'ENERGY', 'MEANDIST', 'MEDIANDIST']


# using distance values
metrics_use_dist_mat = [classifier_distance_cos_mat, IRPR_cos_mat, DC1_cos_mat, DC2_cos_mat, DC3_cos_mat, DC5_cos_mat, DC10_cos_mat, PR1_cos_mat, PR2_cos_mat, PR3_cos_mat, PR5_cos_mat, PR10_cos_mat, Hausdorff_cos_mat, Energy_cos_mat]
metrics_use_dist_mat_names = ['CLASSIFIER', 'IRPR', 'DC_1', 'DC_2', 'DC_3', 'DC_5', 'DC_10', 'PR_1', 'PR_2', 'PR_3', 'PR_5', 'PR_10', 'HAUSDORFF', 'ENERGY']

# using distance values: semantic
metrics_use_dist_mat_semantic = [classifier_cos_semantic, IRPR_cos_semantic, DC1_cos_semantic, DC2_cos_semantic, DC3_cos_semantic, DC5_cos_semantic, DC10_cos_semantic, PR1_cos_semantic, PR2_cos_semantic, PR3_cos_semantic, PR5_cos_semantic, PR10_cos_semantic, Hausdorff_cos_semantic, Energy_cos_semantic]
metrics_use_dist_mat_nonsemantic = [classifier_cos_nonsemantic, IRPR_cos_nonsemantic, DC1_cos_nonsemantic, DC2_cos_nonsemantic, DC3_cos_nonsemantic, DC5_cos_nonsemantic, DC10_cos_nonsemantic, PR1_cos_nonsemantic, PR2_cos_nonsemantic, PR3_cos_nonsemantic, PR5_cos_nonsemantic, PR10_cos_nonsemantic, Hausdorff_cos_nonsemantic, Energy_cos_nonsemantic]

In [ ]:
# run KSC on original corpora using TFIDF (i.e., a non-semantic vector) encoding
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vec = CountVectorizer()
tfidf = TfidfTransformer()

both_data = vec.fit_transform(corpus1 + corpus2).toarray()
both_data = tfidf.fit_transform(both_data).todense()
both_data = np.asarray(both_data).astype(float)

corpus1_tfidf = both_data[:len(corpus1),:]
corpus2_tfidf = both_data[-len(corpus2):,:]

metrics_df, distances_df, _ = runKSC_fixed_sample(base_metrics_nomod, metric_names=base_metrics_names, corpus1=corpus1_tfidf, corpus2=corpus2_tfidf, repetitions=nrepetitions, n=subsample_size, k=15, coverage=True, output=None)
plotKSC(distances_df, boxplot=True, standardized=False, ncolumns=6)
plotKSC(distances_df, boxplot=True, standardized=True, ncolumns=6)

In [ ]:
# use pairwise distance matrix and then apply the distance metric (without binarization)
metrics_df, distances_df, _ = runKSC_fixed_sample(metrics_use_dist_mat, metric_names=metrics_use_dist_mat_names, corpus1=corpus1, corpus2=corpus2, repetitions=nrepetitions, n=subsample_size, k=15, coverage=True, output=None)
plotKSC(distances_df, boxplot=True, standardized=False, ncolumns=7)
plotKSC(distances_df, boxplot=True, standardized=True, ncolumns=7)

In [ ]:
# use semantic binary encoding
metrics_df, distances_df, _ = runKSC_fixed_sample(metrics_use_dist_mat_semantic, metric_names=metrics_use_dist_mat_names, corpus1=corpus1, corpus2=corpus2, repetitions=nrepetitions, n=subsample_size, k=15, coverage=True, output=None)
plotKSC(distances_df, boxplot=True, standardized=False, ncolumns=7)
plotKSC(distances_df, boxplot=True, standardized=True, ncolumns=7)

In [ ]:
# use non-semantic binary encoding to distance matrix and then apply the distance metric
metrics_df, distances_df, _ = runKSC_fixed_sample(metrics_use_dist_mat_nonsemantic, metric_names=metrics_use_dist_mat_names, corpus1=corpus1, corpus2=corpus2, repetitions=nrepetitions, n=subsample_size, k=15, coverage=True, output=None)
plotKSC(distances_df, boxplot=True, standardized=False, ncolumns=7)
plotKSC(distances_df, boxplot=True, standardized=True, ncolumns=7)